In [2]:
import sys
sys.path.append('../../')
from utils import data_loader

In [9]:
db_config = {
    'host': 'localhost',
    'port': 5434,  # Update with your port number
    'dbname': 'atlas_dev',
    'user': 'atlas_driver_offer_bpp_user'
}
db_connection = data_loader.connect_db(db_config)

In [16]:
df = data_loader.load_db_data(db_connection, 'atlas_driver_offer_bpp', 'ride')

In [17]:
df

,id,booking_id,short_id,status,driver_id,otp,tracking_url,fare,traveled_distance,created_at,...,merchant_operating_city_id,number_of_osrm_snap_to_road_calls,safety_alert_triggered,is_free_ride,end_otp,start_odometer_reading_value,end_odometer_reading_value,start_odometer_reading_file_id,end_odometer_reading_file_id,enable_frequent_location_updates
